# RSI momentum trading strategy example

- This is a backtest example notebook
    - Grid search over the hotspot 

# Set up

Set up Trading Strategy data client.


In [3]:
from tradeexecutor.utils.notebook import setup_charting_and_output
from tradingstrategy.client import Client

client = Client.create_jupyter_client()

# Render for Github web viewer
from tradeexecutor.utils.notebook import setup_charting_and_output, OutputMode

setup_charting_and_output(OutputMode.static, image_format="png", width=1500, height=1000)
# setup_charting_and_output(width=1500, height=1000)

Started Trading Strategy in Jupyter notebook environment, configuration is stored in /Users/moo/.tradingstrategy


# Load data

We use Binance data so we get a longer period of data.

In [4]:
import datetime
from tradingstrategy.timebucket import TimeBucket
from tradeexecutor.utils.binance import create_binance_universe

strategy_universe = create_binance_universe(
    ["BTCUSDT", "ETHUSDT"],   # Binance internal tickers later mapped to Trading strategy DEXPair metadata class
    candle_time_bucket=TimeBucket.h8,
    stop_loss_time_bucket=TimeBucket.h1,
    start_at=datetime.datetime(2019, 1, 1),  # Backtest for 5 years data
    end_at=datetime.datetime(2024, 1, 1),
    include_lending=False
)


SyntaxError: '(' was never closed (trading_strategy_universe.py, line 196)

# Show loaded trading universe

Display generic troubleshooting information about the loaded data.

In [ ]:
pairs = strategy_universe.data_universe.pairs  # Trading pairs metadata
candles = strategy_universe.data_universe.candles  # Candles for all trading pairs

print(f"Loaded {candles.get_candle_count():,} candles.")

for pair in pairs.iterate_pairs():
    pair_candles = candles.get_candles_by_pair(pair)
    first_close = pair_candles.iloc[0]["close"]
    first_close_at = pair_candles.index[0]
    print(f"Pair {pair} first close price {first_close} at {first_close_at}")

# Trading algorithm

# Search space

Prepare the parameters we search.

In [ ]:
from tradeexecutor.strategy.cycle import CycleDuration
from pathlib import Path
from tradeexecutor.backtest.grid_search import prepare_grid_combinations

# This is the path where we keep the result files around
storage_folder = Path("/tmp/v13-small-timeframe")

class StrategyParameters:
    cycle_duration = CycleDuration.cycle_1d
    rsi_bars = [8, 9, 10]  # The length of RSI indicator
    eth_btc_rsi_bars = [10, 15]  # The length of ETH/BTC RSI
    rsi_high = [65, 67.5, 70, 72.5] # RSI trigger threshold for decision making
    rsi_low = [55, 57, 60, 62, 65]  # RSI trigger threshold for decision making
    allocation = 0.85 # Allocate 90% of cash to each position
    lookback_candles = 120
    minimum_rebalance_trade_percent = 0.10  # Don't do trades that would have less than 10% impact on the portfolio
    initial_cash = 10_000 # Start with 10k USD
    trailing_stop_loss = [None, 0.8]


combinations = prepare_grid_combinations(StrategyParameters, storage_folder)
print(f"We prepared {len(combinations)} grid search combinations")

# Grid search

Run a grid search on the above function.

In [ ]:
from tradeexecutor.backtest.grid_search import perform_grid_search

grid_search_results = perform_grid_search(
    decide_trades,
    strategy_universe,
    combinations,
    max_workers=8,
    trading_strategy_engine_version="0.4",
    multiprocess=True,
)


# Grid search result table

- Show individual profit and risk for each grid combination

In [ ]:
# Set Jupyter Notebook output mode parameters
from tradeexecutor.analysis.grid_search import analyse_grid_search_result
from tradeexecutor.analysis.grid_search import visualise_table

# Print extension of our backtest
print(f"Grid search combinations available: {len(grid_search_results)}")

table = analyse_grid_search_result(grid_search_results)
visualise_table(table)


# Heatmap

- Verify that results look to cluster around certain values and do not follow a random pattern]

In [ ]:
from tradeexecutor.analysis.grid_search import visualise_heatmap_2d


# RSI length = 8, no stop loss
heatmap_data = table.xs(key=(8, 15, pd.NA), level=("rsi_bars", "eth_btc_rsi_bars", "trailing_stop_loss"))

fig = visualise_heatmap_2d(heatmap_data, "rsi_low", "rsi_high", "CAGR")
display(fig)


In [ ]:
from tradeexecutor.analysis.grid_search import visualise_heatmap_2d


# RSI length = 9, no stop loss
heatmap_data = table.xs(key=(9, 10, pd.NA), level=("rsi_bars", "eth_btc_rsi_bars", "trailing_stop_loss"))

fig = visualise_heatmap_2d(heatmap_data, "rsi_low", "rsi_high", "CAGR")
display(fig)


# Equity curve comparison

- Compare the equity curves of all grid search results
- Show the grid search parameters as the tool tip
- Add buy and hold benchmarks on the plot

In [ ]:
from tradeexecutor.analysis.grid_search import visualise_grid_search_equity_curves
from tradeexecutor.visual.benchmark import create_benchmark_equity_curves

benchmark_indexes = create_benchmark_equity_curves(
    strategy_universe,
    {"BTC": our_pairs[0], "ETH": our_pairs[1]},
    initial_cash=StrategyParameters.initial_cash,
)

# Remove visual clutter
filtered_grid_search_results = [r for r in grid_search_results if r.get_metric("CAGR﹪") >= 0.40]

fig = visualise_grid_search_equity_curves(
    filtered_grid_search_results,
    benchmark_indexes=benchmark_indexes,
    log_y=True,
)
fig.show()